In [30]:
import open3d as o3d
import numpy as np
import cv2
import os
import multiprocessing
from multiprocessing import Pool
import sys
import tqdm

In [13]:

def getCameraIntrinsicMatrix(intrinsicMatrixPath):
    camera_intrinsics = np.loadtxt(intrinsicMatrixPath).astype(np.float64)
    fx = camera_intrinsics[0][0]
    fy = camera_intrinsics[1][1]
    S = camera_intrinsics[0][1]
    cx = camera_intrinsics[0][2]
    cy = camera_intrinsics[1][2]

    camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(width=640, height=480, fx=fx,fy=fy,cx=cx,cy=cy)
    
    return camera_intrinsic

In [3]:
imagepath = "/scratch/small-object-detection/color"
depthpath = "/scratch/small-object-detection/depth"
posepath = "/scratch/small-object-detection/pose"

In [16]:
cameraIntrinsicMatrix = getCameraIntrinsicMatrix("/scratch/small-object-detection/intrinsics/intrinsic_depth.txt")


In [38]:
def registration_result(source, target, transformation):
    print("here")
    target.transform(transformation)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.vstack((np.asarray(source.points), np.asarray(target.points))))
    pcd.colors = o3d.utility.Vector3dVector(np.vstack((np.asarray(source.colors), np.asarray(target.colors))))

    return pcd

In [41]:
def display():
    print("here")

In [50]:
def register_frame(scan):
#     print(f"Starting for {scan}\n")
    global depthpath,imagepath,reg_p2p,transforms,lock
    
    depth_file = depthpath+"/"+scan+'.npy'
    image_file = imagepath+'/'+scan+'.png'

    if(not os.path.exists(depth_file)):
        return

    if(not os.path.exists(image_file)):
        return


    try:
        depth_map = o3d.geometry.Image(np.ascontiguousarray(np.load(depth_file)).astype(np.float32))
        rgb_img = cv2.cvtColor(cv2.imread(image_file),cv2.COLOR_BGR2RGB)
    except:
        print(f"error caused due to {depth_file} or {image_file}")
        return

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(o3d.geometry.Image(rgb_img), o3d.geometry.Image(depth_map),convert_rgb_to_intensity=False)

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cameraIntrinsicMatrix)

    # Transform the point cloud in local frame to global world coordinates
    pose = np.loadtxt(posepath+'/'+scan+'.txt',dtype=np.float32)
    pcd.transform(pose)

    # Take the each 25ht point of the pointcloud
    pcd = pcd.uniform_down_sample(100)

    # Set initial transformation and threshold. May depend on previous params
    threshold = 0.005
    trans_init = np.asarray([[1.0, 0, 0, 0],
                             [0, 1.0, 0, 0],
                             [0, 0, 1.0, 0],
                             [0.0, 0.0, 0.0, 1.0]])
    
    
    lock.acquire()
#     print(f"Acquired Lock for {scan} \n")
    if reg_p2p is not None and np.asarray(reg_p2p.points).shape != 0:
        reg_res = o3d.pipelines.registration.registration_icp(
                reg_p2p, pcd, threshold, trans_init,
                o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
        transforms.append(reg_res.transformation)
        reg_p2p = registration_result(reg_p2p, pcd, reg_res.transformation)
    else:
        reg_p2p = pcd
#     reg_p2p = pcd
#     print(f"Released Lock for {scan} \n")
    lock.release()
#     print(f"Exit for {scan}\n")
    
    sys.stdout.flush()

In [51]:
scans_list = os.listdir(posepath)
scans_list = list(map(lambda x: x.split('.')[0],scans_list))

transforms = list()

reg_p2p = None


lock = multiprocessing.Lock()

with Pool() as pool:
    for _ in tqdm.tqdm(pool.imap_unordered(register_frame, scans_list), total=len(scans_list)):
        pass
    
# with Pool() as pool:
#     pool.map(register_frame,scans_list)

  6%|██████████▌                                                                                                                                                         | 52/808 [00:07<01:47,  7.04it/s]


KeyboardInterrupt: 

In [5]:
print(multiprocessing.cpu_count())

40
